<a href="https://colab.research.google.com/github/LatiefDataVisionary/data-science-capstone-project-college/blob/main/notebooks/02_eda_raw_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Exploratory Data Analysis of Raw Spotify Reviews**

This notebook performs an initial Exploratory Data Analysis (EDA) on the raw dataset of Spotify app reviews. The goal is to understand the basic structure, quality, and characteristics of the data before any preprocessing steps are applied.

## **1. Setup and Library Imports**

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

# Set plot style
sns.set_style('whitegrid')

## 2. Data Loading and Initial Inspection

Loading the dataset from the specified CSV file into a pandas DataFrame.

In [ ]:
# Load the dataset
try:
    raw_df = pd.read_csv('../data/raw/spotify_reviews_raw.csv')
    print("Dataset loaded successfully.")
except FileNotFoundError:
    print("Error: spotify_reviews_raw.csv not found. Please ensure the file is in the correct directory.")
    raw_df = None # Set df to None to avoid errors in subsequent cells

Displaying the first 5 rows of the DataFrame to get a glimpse of the data structure and content.

In [ ]:
if raw_df is not None:
    display(raw_df.head())

Printing concise information about the DataFrame, including the index dtype and column dtypes, non-null values, and memory usage. This helps in understanding the data types and identifying potential missing values.

In [ ]:
if raw_df is not None:
    raw_df.info()

Generating descriptive statistics for the numerical columns in the DataFrame. This provides insights into the central tendency, dispersion, and shape of the distribution of numerical data.

In [ ]:
if raw_df is not None:
    display(raw_df.describe())

## 3. Data Quality Check

Checking for missing values in each column and summing them up to get a total count of missing entries per column.

In [ ]:
if raw_df is not None:
    missing_values = raw_df.isnull().sum()
    print("Missing values per column:")
    print(missing_values)

Checking for and counting duplicate rows in the DataFrame. Duplicate rows can skew analysis and should be identified.

In [ ]:
if raw_df is not None:
    duplicate_rows = raw_df.duplicated().sum()
    print(f"\nNumber of duplicate rows: {duplicate_rows}")

**Summary of Data Quality:**

Based on the checks above:
- We can see the number of missing values in each column.
- We have identified the total count of duplicate rows.

This initial assessment helps us understand the cleanliness of the raw data and highlights areas that may require attention during preprocessing.

## 4. Analysis of Review Ratings (score)

Visualizing the distribution of review scores using a count plot to understand the frequency of each rating (1 to 5 stars).

In [ ]:
if raw_df is not None:
    plt.figure(figsize=(8, 6))
    sns.countplot(data=raw_df, x='score', palette='viridis')
    plt.title('Distribution of Review Scores')
    plt.xlabel('Score')
    plt.ylabel('Number of Reviews')
    plt.show()

**Analysis of Score Distribution:**

The count plot reveals the distribution of star ratings. We can observe which scores are most frequent. This gives us an initial idea of the overall sentiment of the reviews and the balance of the dataset. For example, if one score significantly outweighs others, the dataset is skewed.

## 5. Analysis of Review Text (content)

### Subsection 5.1: Review Length Analysis

Creating new columns for character length and word count of each review in the `content` column.

In [ ]:
if raw_df is not None and 'content' in raw_df.columns:
    raw_df['char_length'] = raw_df['content'].astype(str).apply(len)
    raw_df['word_count'] = raw_df['content'].astype(str).apply(lambda x: len(x.split()))
    print("Character length and word count columns added.")

Generating histograms to visualize the distribution of review lengths based on character count and word count.

In [ ]:
if raw_df is not None and 'char_length' in raw_df.columns and 'word_count' in raw_df.columns:
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))

    sns.histplot(data=raw_df, x='char_length', bins=50, ax=axes[0], kde=True)
    axes[0].set_title('Distribution of Review Character Length')
    axes[0].set_xlabel('Character Length')
    axes[0].set_ylabel('Frequency')

    sns.histplot(data=raw_df, x='word_count', bins=50, ax=axes[1], kde=True)
    axes[1].set_title('Distribution of Review Word Count')
    axes[1].set_xlabel('Word Count')
    axes[1].set_ylabel('Frequency')

    plt.tight_layout()
    plt.show()

**Analysis of Review Length:**

The histograms show the distribution of how long reviews are, both in terms of the number of characters and the number of words. This helps us understand the typical length of reviews and identify any outliers (very short or very long reviews). Most reviews appear to be concentrated within a certain range of words/characters.

### Subsection 5.2: Word Cloud of Raw Text

Generating a Word Cloud from the raw review text to visually represent the most frequent words. Note that this is based on unprocessed text and will include stopwords and other noise.

In [ ]:
if raw_df is not None and 'content' in raw_df.columns:
    # Combine all reviews into a single string
    all_reviews = " ".join(review for review in raw_df['content'].astype(str))

    # Generate word cloud
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(all_reviews)

    # Display the word cloud
    plt.figure(figsize=(10, 7))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.title('Word Cloud of Raw Review Text')
    plt.show()

**Analysis of Word Cloud:**

The word cloud provides a quick visual overview of the most frequently occurring words in the raw review text. As expected with unprocessed text, it contains many common words (stopwords) and potential noise. However, it gives a raw first impression of the dominant themes and keywords present in the reviews before any cleaning or preprocessing.

## 6. Temporal Analysis (Review Timestamps)

Converting the `at` column to a datetime object to enable time-based analysis. Errors in conversion will be coerced to `NaT` (Not a Time).

In [ ]:
if raw_df is not None and 'at' in raw_df.columns:
    raw_df['at'] = pd.to_datetime(raw_df['at'], errors='coerce')
    print(" 'at' column converted to datetime objects.")
    # Drop rows where datetime conversion failed
    raw_df.dropna(subset=['at'], inplace=True)
    print(f"Dropped rows with invalid datetime values. Remaining rows: {len(raw_df)}")

Plotting the number of reviews submitted over time, grouped by month, to identify any trends or patterns in review activity.

In [ ]:
if raw_df is not None and 'at' in raw_df.columns:
    # Extract month and year
    raw_df['review_month'] = raw_df['at'].dt.to_period('M')

    # Group by month and count reviews
    monthly_reviews = raw_df.groupby('review_month').size()

    # Plot the time series
    plt.figure(figsize=(12, 6))
    monthly_reviews.plot()
    plt.title('Number of Reviews Over Time (Monthly)')
    plt.xlabel('Date')
    plt.ylabel('Number of Reviews')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

**Analysis of Temporal Trends:**

The time series plot shows how the number of reviews has changed over the months. We can observe trends such as periods of increased or decreased review activity. Spikes in reviews might correlate with significant events like app updates, marketing campaigns, or service outages. A general upward or downward trend could indicate changes in user engagement over time.

## 7. Conclusion & Next Steps

**Conclusion:**

This initial EDA on the raw Spotify reviews dataset has provided valuable insights into its structure, quality, and content. We have examined:
- The dataset's basic information and structure.
- The presence of missing values and duplicate rows.
- The distribution of review scores, indicating the overall sentiment.
- The characteristics of the review text, including length and the most frequent raw words.
- The temporal trends in review submission.

The raw text data, as seen in the word cloud, is noisy and requires significant cleaning and preprocessing before it can be effectively used for tasks like sentiment analysis or topic modeling.

**Next Steps:**

The logical next step in this project, which will be covered in the notebook `03_data_preprocessing_and_cleaning.ipynb`, is to clean and preprocess the raw text data. This will involve steps such as:
- Handling missing values and duplicate rows (if not already done).
- Text cleaning (e.g., removing punctuation, special characters, numbers).
- Lowercasing the text.
- Tokenization.
- Removing stopwords.
- Stemming or lemmatization.
- Handling bilingual text (if necessary).

These preprocessing steps are crucial to transform the raw review text into a format suitable for further analysis and machine learning models.

In [2]:
data1 = 'https://raw.githubusercontent.com/LatiefDataVisionary/data-science-capstone-project-college/refs/heads/main/data/raw/spotify_reviews_en_au_1.csv'
data2 = 'https://raw.githubusercontent.com/LatiefDataVisionary/data-science-capstone-project-college/refs/heads/main/data/raw/spotify_reviews_en_au_2.csv'
data3 = 'https://raw.githubusercontent.com/LatiefDataVisionary/data-science-capstone-project-college/refs/heads/main/data/raw/spotify_reviews_en_uk_1.csv'
data4 = 'https://raw.githubusercontent.com/LatiefDataVisionary/data-science-capstone-project-college/refs/heads/main/data/raw/spotify_reviews_en_uk_2.csv'
data5 = 'https://raw.githubusercontent.com/LatiefDataVisionary/data-science-capstone-project-college/refs/heads/main/data/raw/spotify_reviews_en_us_1.csv'
data6 = 'https://raw.githubusercontent.com/LatiefDataVisionary/data-science-capstone-project-college/refs/heads/main/data/raw/spotify_reviews_en_us_2.csv'